In [4]:
# mars rover action plan
# visualization in Nengo
# trajectory following w/ adaptive control in Nengo
# interface w/ pioneer in Coppelia

In [5]:
import numpy as np

from nengo_gui.ipython import IPythonViz
import nengo


def rectangle(x=0, y=0, width=1, height=1, color='white', outline_color='black', outline_width=.1):
    return f'<rect width="{width}" height="{height}" style="fill:{color};stroke:{outline_color};stroke-width:{outline_width}" transform="translate({x},{y})"/>'

def circle(x=0, y=0, r=30, fill='purple', stroke = 'white'):
    return f'<circle cx="{x}" cy="{y}" r="{r}" fill="{fill}" stroke="{stroke}"/>'

def line(x1 = 100, y1 = 100, x2 = 900, y2 = 900, color='black', width=5):
    return f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" style="stroke:{color};stroke-width:{width}"/>'

def canvas(shape_list, width=10, height=10):
    svg = f'<svg width="100%%" height="100%%" viewbox="0 0 {width} {height}">'
    for shape in shape_list:
        svg += shape
    svg += '</svg>'
    return svg

class PathEnvironment(object):
    '''
    Defines a dynamic environment for a path-following robot.
        Arguments:
            size : size of square environment. Default = 1000.
            dt   : size of simulation timestep, in seconds. Default is 0.01.

        Methods:
            __call__:
                Updates the environment, defined by the desired path.
                Returns distance of the agent from the path. 

    '''
    def __init__(self,size=1000,dt=0.01):
        self.size = size
        self.dt = dt
        self._nengo_html_ = ''
        
        # define a line in the environment to follow
        self.x1 = 100
        self.y1 = 100
        self.x2 = 900
        self.y2 = 900
        
        # agent position
        self.ax = 335
        self.ay = 603
        
    def __call__(self,t,input_):
        
        p2x = input_[0]
        p2y = input_[1]
        avx = input_[2]
        avy = input_[3]
        
        self.x2 = p2x
        self.y2 = p2y
        
        self.ax += avx * self.dt
        self.ay += avy * self.dt
        
        self.render()
        
        # compute error vector to the closest point on the path
        m = (self.y2-self.y1)/(self.x2-self.x1)
        b = self.y1 - m*self.x1
        
        m_perp = -1/m
        b_perp = self.ay - m_perp*self.ax
        
        x_soln = (b-b_perp)/(m_perp-m)
        y_soln = m * x_soln + b
        
        error = np.array([y_soln-self.ay,x_soln-self.ax])
        
        return error
    
    def render(self):
        # generate the shape list for html rendering
        shape_list = []
        shape_list.append( line( x1 = self.x1, y1 = self.y1, x2 = self.x2, y2 = self.y2 ) )
        shape_list.append( circle( x = self.ax, y = self.ay ) )

        # generate the html from the list of shapes
        self._nengo_html_ = canvas(shape_list, width = self.size,height=self.size)

size = 1000
dt = 0.01
kp = 0.2

with nengo.Network() as model:
    
    # allow the user to play god and shape the environment
    path_end_point = nengo.Node([900, 900])
    
    # create a node to accept the desired state
    # v_stim = nengo.Node( [0,0] )#output = lambda t: [np.sin(2*np.pi*t)*100,np.cos(2*np.pi*t)*100] )
    
    # create the environment
    env = nengo.Node( PathEnvironment( size = size, dt = dt ), size_in = 4, size_out = 2 )
    nengo.Connection( path_end_point, env[:2], synapse = None)
    # nengo.Connection( v_stim, env[2:] )
    
    # compute the error -- proportional only
    def control( error ):
        print(error)
        return - kp * error
    
    controller_pop = nengo.Ensemble(
                        n_neurons = 1000,
                        dimensions = 2,
                        radius = 500,
                        )
    nengo.Connection( env, controller_pop )
    nengo.Connection( controller_pop, env[2:], function = control)
    
    #controller = nengo.Node( output = control, size_in =  2, size_out = 2 )
    # pass the error computed by the environment to the controller
    #nengo.Connection( env, controller, synapse = None )
    #nengo.Connection( controller, env[2:], synapse = 0 )

In [6]:
import nengo_gui.jupyter
nengo_gui.jupyter.InlineGUI(model, cfg='line_prop_control.cfg')